<a href="https://colab.research.google.com/github/noahfavreau/nasa-space-apps-2025/blob/main/model_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install joblib matplotlib numpy optuna pandas lightgbm xgboost catboost pytorch-tabnet scikit-learn

In [ ]:
import json
from functools import partial
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import pickle
import random

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier, plot_importance

RANDOM_SEED = 67


In [ ]:
ARTIFACT_ROOT = Path("artifacts")


def ensure_dir(path: Path) -> Path:
    path.mkdir(parents=True, exist_ok=True)
    return path


def reshape_for_meta(array: np.ndarray) -> np.ndarray:
    array = np.asarray(array)
    if array.ndim == 1:
        return array.reshape(-1, 1)
    return array


def persist_model(model, destination: Path, model_name: str) -> Path:
    ensure_dir(destination.parent)
    if model_name == "lightgbm":
        file_path = destination.with_suffix(".txt")
        model.save_model(str(file_path))
    elif model_name == "catboost":
        file_path = destination.with_suffix(".cbm")
        model.save_model(str(file_path))
    elif model_name == "xgboost":
        file_path = destination.with_suffix(".json")
        model.save_model(str(file_path))
    elif model_name == "tabnet":
        base_path = str(destination)
        if base_path.endswith(".zip"):
            base_path = base_path[:-4]
        model.save_model(base_path)
        file_path = Path(f"{base_path}.zip")
    else:
        file_path = destination.with_suffix(".pkl")
        with open(file_path, "wb") as fp:
            pickle.dump(model, fp)
    return file_path


def lightgbm_predict_proba(model: lgb.Booster, data):
    best_iter = model.best_iteration or model.current_iteration()
    return model.predict(data, num_iteration=best_iter)


def compute_fold_sample_weight(y_values):
    return compute_sample_weight(class_weight="balanced", y=y_values)


In [ ]:
def train_catboost(X_train, X_val, y_train, y_val, params, fold, sample_weight=None):
    model_params = {**params}
    model_params.setdefault("loss_function", "MultiClass")
    model_params.setdefault("eval_metric", "MultiClass")
    if "task_type" not in model_params:
        model_params["task_type"] = "CPU"
    if model_params.get("task_type") == "GPU" and "devices" not in model_params:
        model_params["devices"] = "0"
    model_params.setdefault("verbose", 0)
    model_params["random_state"] = RANDOM_SEED + fold

    train_pool = Pool(X_train, y_train, weight=sample_weight)
    val_pool = Pool(X_val, y_val)

    model = CatBoostClassifier(**model_params)
    model.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=20,
        use_best_model=True,
        verbose=False,
    )

    val_proba = model.predict_proba(X_val)
    val_preds = np.argmax(val_proba, axis=1)
    accuracy = accuracy_score(y_val, val_preds)

    return model, val_proba, accuracy


In [ ]:
def train_lgb(X_train, X_val, y_train, y_val, params, fold, sample_weight=None, num_classes=3):
    model_params = {**params}
    model_params.setdefault("objective", "multiclass")
    model_params.setdefault("num_class", num_classes)
    model_params.setdefault("metric", "multi_logloss")
    model_params.setdefault("verbose", -1)
    model_params["seed"] = RANDOM_SEED + fold

    train_data = lgb.Dataset(X_train, label=y_train, weight=sample_weight)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    model = lgb.train(
        model_params,
        train_data,
        num_boost_round=200,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=20)],
        verbose_eval=False,
    )

    val_proba = lightgbm_predict_proba(model, X_val)
    val_preds = np.argmax(val_proba, axis=1)
    accuracy = accuracy_score(y_val, val_preds)

    return model, val_proba, accuracy


In [ ]:
def train_xgb(X_train, X_val, y_train, y_val, params, fold, sample_weight=None, num_classes=3):
    model_params = {**params}
    model_params.setdefault("objective", "multi:softprob")
    model_params.setdefault("num_class", num_classes)
    model_params.setdefault("eval_metric", "mlogloss")
    model_params.setdefault("verbosity", 0)
    model_params["random_state"] = RANDOM_SEED + fold
    model_params["use_label_encoder"] = False

    model = XGBClassifier(**model_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        sample_weight=sample_weight,
        early_stopping_rounds=20,
        verbose=False,
    )

    val_proba = model.predict_proba(X_val)
    val_preds = np.argmax(val_proba, axis=1)
    accuracy = accuracy_score(y_val, val_preds)

    return model, val_proba, accuracy


In [ ]:
def train_tabnet(X_train, X_val, y_train, y_val, params, fold, sample_weight=None):
    model_params = {**params}
    model_params.setdefault("seed", RANDOM_SEED + fold)
    model_params.setdefault("verbose", 0)

    model = TabNetClassifier(**model_params)
    model.fit(
        X_train.values,
        y_train.values,
        eval_set=[(X_val.values, y_val.values)],
        eval_metric=["accuracy"],
        max_epochs=200,
        patience=20,
        batch_size=1024,
        virtual_batch_size=128,
        weights=sample_weight,
    )

    val_proba = model.predict_proba(X_val.values)
    val_preds = np.argmax(val_proba, axis=1)
    accuracy = accuracy_score(y_val, val_preds)

    return model, val_proba, accuracy


In [ ]:
def objective_catboost(trial, X, y, n_splits=5, random_seed=RANDOM_SEED):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "task_type": trial.suggest_categorical("task_type", ["CPU", "GPU"]),
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "random_seed": random_seed,
        "verbose": 0,
    }
    if params["task_type"] == "GPU":
        params["devices"] = "0"

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    accuracies = []

    for train_idx, valid_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[valid_idx]
        train_weights = compute_fold_sample_weight(y_train)

        model = CatBoostClassifier(**params)
        model.fit(
            X_train,
            y_train,
            sample_weight=train_weights,
            eval_set=(X_val, y_val),
            early_stopping_rounds=20,
            verbose=False,
        )
        preds = model.predict(X_val)
        accuracies.append(accuracy_score(y_val, preds))

    return float(np.mean(accuracies))


In [ ]:
def objective_lgb(trial, X, y, num_classes, n_splits=5, random_seed=RANDOM_SEED):
    params = {
        "objective": "multiclass",
        "num_class": num_classes,
        "metric": "multi_logloss",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "verbose": -1,
        "seed": random_seed,
    }

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    accuracies = []

    for train_idx, valid_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[valid_idx]
        train_weights = compute_fold_sample_weight(y_train)

        train_data = lgb.Dataset(X_train, label=y_train, weight=train_weights)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            callbacks=[lgb.early_stopping(stopping_rounds=20)],
            verbose_eval=False,
        )

        preds = np.argmax(lightgbm_predict_proba(model, X_val), axis=1)
        accuracies.append(accuracy_score(y_val, preds))

    return float(np.mean(accuracies))


In [ ]:
def objective_xgb(trial, X, y, num_classes, n_splits=5, random_seed=RANDOM_SEED):
    params = {
        "objective": "multi:softprob",
        "num_class": num_classes,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "random_state": random_seed,
        "verbosity": 0,
        "use_label_encoder": False,
    }

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    accuracies = []

    for train_idx, valid_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[valid_idx]
        train_weights = compute_fold_sample_weight(y_train)

        model = XGBClassifier(**params)
        model.fit(
            X_train,
            y_train,
            sample_weight=train_weights,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=20,
            verbose=False,
        )
        preds = model.predict(X_val)
        accuracies.append(accuracy_score(y_val, preds))

    return float(np.mean(accuracies))


In [ ]:
def objective_tabnet(trial, X, y, n_splits=5, random_seed=RANDOM_SEED):
    params = {
        "n_d": trial.suggest_int("n_d", 8, 64, step=8),
        "n_a": trial.suggest_int("n_a", 8, 64, step=8),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0, step=0.1),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True),
        "optimizer_fn": trial.suggest_categorical("optimizer_fn", ["adam", "adamw"]),
        "optimizer_params": {"lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True)},
        "momentum": trial.suggest_float("momentum", 0.01, 0.4),
        "seed": random_seed,
        "verbose": 0,
    }

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    accuracies = []

    for train_idx, valid_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[valid_idx]
        train_weights = compute_fold_sample_weight(y_train)

        model = TabNetClassifier(**params)
        model.fit(
            X_train.values,
            y_train.values,
            eval_set=[(X_val.values, y_val.values)],
            eval_metric=["accuracy"],
            max_epochs=200,
            patience=20,
            batch_size=1024,
            virtual_batch_size=128,
            weights=train_weights,
        )

        preds = model.predict(X_val.values)
        accuracies.append(accuracy_score(y_val, preds))

    return float(np.mean(accuracies))


In [ ]:
def run_stacking_pipeline(
    X,
    y,
    *,
    test_size=0.2,
    n_trials=30,
    base_cv_splits=10,
    optuna_cv_splits=5,
    artifact_root: Path = ARTIFACT_ROOT,
    random_seed: int = RANDOM_SEED,
):
    if not isinstance(X, pd.DataFrame):
        X_df = pd.DataFrame(X)
    else:
        X_df = X.copy()

    if isinstance(y, pd.DataFrame):
        if y.shape[1] != 1:
            raise ValueError("y must be a 1-D vector.")
        y_series = y.iloc[:, 0].copy()
    elif isinstance(y, pd.Series):
        y_series = y.copy()
    else:
        y_series = pd.Series(y)

    if X_df.shape[0] != len(y_series):
        raise ValueError("X and y must have the same number of rows.")
    if len(np.unique(y_series)) < 2:
        raise ValueError("y must contain at least two classes.")

    np.random.seed(random_seed)
    artifact_root = ensure_dir(Path(artifact_root))

    X_train, X_test, y_train, y_test = train_test_split(
        X_df,
        y_series,
        test_size=test_size,
        stratify=y_series,
        random_state=random_seed,
    )

    num_classes = len(np.unique(y_train))

    objective_wrappers = {
        "catboost": partial(
            objective_catboost,
            X=X_train,
            y=y_train,
            n_splits=optuna_cv_splits,
            random_seed=random_seed,
        ),
        "lightgbm": partial(
            objective_lgb,
            X=X_train,
            y=y_train,
            num_classes=num_classes,
            n_splits=optuna_cv_splits,
            random_seed=random_seed,
        ),
        "xgboost": partial(
            objective_xgb,
            X=X_train,
            y=y_train,
            num_classes=num_classes,
            n_splits=optuna_cv_splits,
            random_seed=random_seed,
        ),
        "tabnet": partial(
            objective_tabnet,
            X=X_train,
            y=y_train,
            n_splits=optuna_cv_splits,
            random_seed=random_seed,
        ),
    }

    best_params = {}
    for model_name, objective_fn in objective_wrappers.items():
        study = optuna.create_study(direction="maximize")
        study.optimize(objective_fn, n_trials=n_trials)
        best_params[model_name] = study.best_params
        print(f"{model_name.title()} best params: {study.best_params}")

    with open(artifact_root / "best_params.json", "w", encoding="utf-8") as fp:
        json.dump(best_params, fp, indent=2)

    skf = StratifiedKFold(n_splits=base_cv_splits, shuffle=True, random_state=random_seed)

    oof_predictions = {
        "catboost": np.zeros((len(y_train), num_classes)),
        "lightgbm": np.zeros((len(y_train), num_classes)),
        "xgboost": np.zeros((len(y_train), num_classes)),
        "tabnet": np.zeros((len(y_train), num_classes)),
    }
    test_predictions = {
        "catboost": np.zeros((len(y_test), num_classes)),
        "lightgbm": np.zeros((len(y_test), num_classes)),
        "xgboost": np.zeros((len(y_test), num_classes)),
        "tabnet": np.zeros((len(y_test), num_classes)),
    }
    fold_accuracies = {model: [] for model in oof_predictions}
    saved_model_paths = {model: [] for model in oof_predictions}

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), start=1):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        train_weights = compute_fold_sample_weight(y_tr)

        cb_model, cb_val_proba, cb_acc = train_catboost(
            X_tr,
            X_val,
            y_tr,
            y_val,
            params=best_params["catboost"],
            fold=fold,
            sample_weight=train_weights,
        )
        oof_predictions["catboost"][val_idx] = cb_val_proba
        test_predictions["catboost"] += cb_model.predict_proba(X_test)
        fold_accuracies["catboost"].append(cb_acc)
        saved_model_paths["catboost"].append(
            str(persist_model(cb_model, artifact_root / "catboost" / f"fold_{fold}", "catboost"))
        )

        lgb_model, lgb_val_proba, lgb_acc = train_lgb(
            X_tr,
            X_val,
            y_tr,
            y_val,
            params=best_params["lightgbm"],
            fold=fold,
            sample_weight=train_weights,
            num_classes=num_classes,
        )
        oof_predictions["lightgbm"][val_idx] = lgb_val_proba
        test_predictions["lightgbm"] += lightgbm_predict_proba(lgb_model, X_test)
        fold_accuracies["lightgbm"].append(lgb_acc)
        saved_model_paths["lightgbm"].append(
            str(persist_model(lgb_model, artifact_root / "lightgbm" / f"fold_{fold}", "lightgbm"))
        )

        xgb_model, xgb_val_proba, xgb_acc = train_xgb(
            X_tr,
            X_val,
            y_tr,
            y_val,
            params=best_params["xgboost"],
            fold=fold,
            sample_weight=train_weights,
            num_classes=num_classes,
        )
        oof_predictions["xgboost"][val_idx] = xgb_val_proba
        test_predictions["xgboost"] += xgb_model.predict_proba(X_test)
        fold_accuracies["xgboost"].append(xgb_acc)
        saved_model_paths["xgboost"].append(
            str(persist_model(xgb_model, artifact_root / "xgboost" / f"fold_{fold}", "xgboost"))
        )

        tabnet_model, tabnet_val_proba, tabnet_acc = train_tabnet(
            X_tr,
            X_val,
            y_tr,
            y_val,
            params=best_params["tabnet"],
            fold=fold,
            sample_weight=train_weights,
        )
        oof_predictions["tabnet"][val_idx] = tabnet_val_proba
        test_predictions["tabnet"] += tabnet_model.predict_proba(X_test.values)
        fold_accuracies["tabnet"].append(tabnet_acc)
        saved_model_paths["tabnet"].append(
            str(persist_model(tabnet_model, artifact_root / "tabnet" / f"fold_{fold}", "tabnet"))
        )

        print(f"Fold {fold} complete.")

    for model_name in test_predictions:
        test_predictions[model_name] /= base_cv_splits

    meta_features_train = np.hstack([reshape_for_meta(oof_predictions[name]) for name in oof_predictions])
    meta_features_test = np.hstack([reshape_for_meta(test_predictions[name]) for name in test_predictions])

    meta_model = LogisticRegressionCV(
        cv=base_cv_splits,
        multi_class="multinomial",
        max_iter=1000,
        class_weight="balanced",
        random_state=random_seed,
    )
    meta_model.fit(meta_features_train, y_train)

    meta_train_preds = meta_model.predict(meta_features_train)
    meta_train_acc = accuracy_score(y_train, meta_train_preds)

    meta_test_preds = meta_model.predict(meta_features_test)
    meta_test_acc = accuracy_score(y_test, meta_test_preds)
    meta_test_proba = meta_model.predict_proba(meta_features_test)
    meta_report = classification_report(y_test, meta_test_preds, digits=4)

    meta_model_path = artifact_root / "meta_model.joblib"
    joblib.dump(meta_model, meta_model_path)

    predictions_df = pd.DataFrame(
        {
            "y_true": y_test.values,
            "y_pred": meta_test_preds,
        }
    )
    predictions_path = artifact_root / "meta_model_test_predictions.csv"
    predictions_df.to_csv(predictions_path, index=False)

    np.save(artifact_root / "meta_features_train.npy", meta_features_train)
    np.save(artifact_root / "meta_features_test.npy", meta_features_test)
    np.save(artifact_root / "meta_test_proba.npy", meta_test_proba)

    summary = {
        "best_params": best_params,
        "fold_metrics": {
            name: {
                "mean_accuracy": float(np.mean(scores)),
                "std_accuracy": float(np.std(scores)),
            }
            for name, scores in fold_accuracies.items()
        },
        "meta_train_accuracy": float(meta_train_acc),
        "meta_test_accuracy": float(meta_test_acc),
        "classification_report": meta_report,
        "saved_models": saved_model_paths,
        "meta_model_path": str(meta_model_path),
        "test_predictions_path": str(predictions_path),
    }

    return summary


In [ ]:
data_path = Path("dataset/combined_imputed_values.csv")

if data_path.exists():
    df = pd.read_csv(data_path)
    if "disposition" not in df.columns:
        raise KeyError("Column 'disposition' is missing from the dataset.")
    X = df.drop(columns=["disposition"])
    y = df["disposition"]
    print(f"Loaded dataset with shape {df.shape}.")
else:
    print("Dataset 'combined_imputed_values.csv' not found. Define X and y manually.")


In [ ]:
if "X" in globals() and "y" in globals():
    pipeline_results = run_stacking_pipeline(
        X,
        y,
        test_size=0.2,
        n_trials=30,
        base_cv_splits=10,
        optuna_cv_splits=5,
        artifact_root=ARTIFACT_ROOT,
        random_seed=RANDOM_SEED,
    )
    pipeline_results
else:
    print("Define X (features) and y (target) before running the stacking pipeline.")
